In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
#Import Libraries
import numpy as np
import pylab as plt
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.metrics import explained_variance_score

import warnings
warnings.filterwarnings("ignore")

In [ ]:
#Loading test and train Data
import pandas as pd
gender_submission = pd.read_csv("../input/titanic/gender_submission.csv")
test = pd.read_csv("../input/titanic/test.csv")
train = pd.read_csv("../input/titanic/train.csv")

In [ ]:
train.head()

In [ ]:
train.dtypes

In [ ]:
train.describe()

******Exploratory Data Analysis of Survivors

In [ ]:
sns.barplot(x="Sex",y="Survived",hue="Pclass",data=train)

In [ ]:
#Survival of Passengers in each class
survivors=train.groupby('Pclass')['Survived'].agg(sum)

In [ ]:
total_passengers=train.groupby('Pclass')['PassengerId'].count()

In [ ]:
survivor_percentage=survivors/total_passengers

In [ ]:
#Visualizing total number of survivors
fig=plt.figure()
ax=fig.add_subplot(111)
rect=ax.bar(survivors.index.values.tolist(),survivors,color='blue',width=0.5)
ax.set_ylabel('No of survivors')
ax.set_title('Number of survivors based on class')
xTickMarks=survivors.index.values.tolist()
ax.set_xticks(survivors.index.values.tolist())
xtickNames = ax.set_xticklabels(xTickMarks)
plt.setp(xtickNames,fontsize=20)
plt.show()

In [ ]:
#Visualizing total number of survivors
fig=plt.figure()
ax=fig.add_subplot(111)
rect=ax.bar(survivor_percentage.index.values.tolist(),survivor_percentage,color='blue',width=0.5)
ax.set_ylabel('Percentage of survivors')
ax.set_title('Percentage of survivors based on class')
xTickMarks=survivors.index.values.tolist()
ax.set_xticks(survivors.index.values.tolist())
xtickNames=ax.set_xticklabels(xTickMarks)
plt.setp(xtickNames,fontsize=20)
plt.show()

In [ ]:
,#Visualize by age
group_by_age=pd.cut(train["Age"],np.arange(0,90,10))
age_grouping=train.groupby(group_by_age).mean()
age_grouping['Survived'].plot.bar()

In [ ]:
,#Visualize by age
group_by_age=pd.cut(train["Age"],np.arange(0,90,10))
age_grouping=train.groupby(group_by_age).mean()
age_grouping['Survived'].plot.pie()

In [ ]:
#Visualize with a facegrid
g=sns.FacetGrid(train,col="Pclass")
g.map(sns.boxplot,"Sex","Age",palette="Set3")

****Exploratory Data analysis of Non-Survivors

In [ ]:
non_survivors=train[(train['SibSp']>0) |(train['Parch']>0 &(train['Survived']==0))].groupby('Pclass')['Survived'].agg('count')

total_passengers=train.groupby('Pclass')['PassengerId'].count()

In [ ]:
non_survivor_percentage=non_survivors/total_passengers

In [ ]:
#Total number of non survivors with family based on class
fig=plt.figure()
fig=plt.figure()
ax=fig.add_subplot(111)
rect=ax.bar(non_survivors.index.values.tolist(),non_survivors,color='blue',width=0.5)
ax.set_ylabel('No of non survivors')
ax.set_title('Total No of non survivors based on family class')
xTickMarks=non_survivors.index.values.tolist()
ax.set_xticks(non_survivors.index.values.tolist())
xtickNames=ax.set_xticklabels(xTickMarks)
plt.setp(xtickNames,fontsize=20)
plt.show()

In [ ]:
#Percentage of non survivors with family based on class
fig=plt.figure()
fig=plt.figure()
ax=fig.add_subplot(111)
rect=ax.bar(non_survivor_percentage.index.values.tolist(),non_survivor_percentage,color='blue',width=0.5)
ax.set_ylabel('Percentage of non survivors')
ax.set_title('Total Percentage of non survivors based on family class')
xTickMarks=non_survivor_percentage.index.values.tolist()
ax.set_xticks(non_survivor_percentage.index.values.tolist())
xtickNames=ax.set_xticklabels(xTickMarks)
plt.setp(xtickNames,fontsize=20)
plt.show()

**Data Preprocessing for Machine Learning

In [ ]:
train.info()

In [ ]:
train.isnull().sum()

In [ ]:
train=train.drop(["Cabin"],axis=1)
train.head()

In [ ]:
test.head()

In [ ]:
data=pd.concat([train,test],keys=['x','y'])
data.head()

In [ ]:
data.shape

In [ ]:
data.isnull().sum()

In [ ]:
num_col=data._get_numeric_data().columns.tolist()
num_col

In [ ]:
cat_col=data.select_dtypes(exclude=["number"])
cat_col

In [ ]:
for col in num_col:
    data[col].fillna(data[col].mean(),inplace=True)

In [ ]:
for col in cat_col:
    data[col].fillna(data[col].mode(),inplace=True)

In [ ]:
for i in cat_col:
    print(data[i].value_counts())

In [ ]:
data.shape

In [ ]:
data=data.drop(["Name","Ticket","Cabin"],axis=1)

In [ ]:
X=pd.get_dummies(data)
X.shape

In [ ]:
train_data=X.loc["x"]
train_data.shape

In [ ]:
test_data=X.loc["y"]
test_data.shape

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scalerX =MinMaxScaler(feature_range=(0, 1))
X[X.columns]=scalerX.fit(X[X.columns])

In [ ]:
train_data.head()

In [ ]:
train_data.isnull().sum()

**Feaure Selection**

In [ ]:
fig,ax=plt.subplots()
ax.scatter(x=train_data['Age'],y=train_data['Pclass'])
plt.ylabel('Pclass',fontsize=13)
plt.xlabel('Age',fontsize=13)
plt.show()

In [ ]:
plt.figure(figsize=(12,8))
plt.title('Titanic Correlation of Features',y=1.05,size=15)
sns.heatmap(train_data.corr(), linewidths=0.1 ,vmax=1.0,square=True,linecolor='white',annot=True)

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score


In [ ]:
test = pd.read_csv("../input/titanic/test.csv")
train = pd.read_csv("../input/titanic/train.csv")

In [ ]:
print('Shape of training data:',train_data.shape)
print('Shape of testing data:',train_data.shape)


In [ ]:
train_x=train_data.drop(columns=['Survived'],axis=1)
train_y=train_data['Survived']

In [ ]:
test_x=train_data.drop(columns=['Survived'],axis=1)
test_y=train_data['Survived']

In [ ]:
model=XGBClassifier()

In [ ]:
model.fit(train_x,train_y)

In [ ]:
predict_train=model.predict(train_x)
print('\nTarget on train data',predict_train)

In [ ]:
accuracy_train=accuracy_score(train_y,predict_train)
print('\naccuracy_score  on train dataset:',accuracy_train)

In [ ]:
predict_test=model.predict(train_x)
print('\nTarget on test data',predict_test)

In [ ]:
accuracy_test=accuracy_score(test_y,predict_test)
print('\naccuracy_score on test dataset:',accuracy_test)

**Submission

In [ ]:
train_data.head()

In [ ]:
gender_submission.head()

In [ ]:
gender_submission3=pd.DataFrame({ "PassengerId":train_data["PassengerId"],
                               "Survived":predict_test})

In [ ]:
gender_submission3['Survived']=predict_test
gender_submission3.to_csv('xgb_submission3.csv',index=False)
gender_submission3.head(10)

In [ ]:
df_unstacked = df_stacked.unstack() 
print(df_unstacked.head(417)) 

In [ ]:
gender_submission4=pd.DataFrame({ "PassengerId":df_unstacked["PassengerId"],
                               "Survived":predict_test})

In [ ]:
gender_submission4.to_csv('xgb_submission5.csv',index=False)

In [ ]:
gender_submission4.dtypes

In [ ]:
gender_submission4['PassengerId']=gender_submission4['PassengerId'].astype(int)
gender_submission4['Survived']=gender_submission4['Survived'].astype(int)

In [ ]:
gender_submission4.dtypes

In [ ]:
gender_submission4['PassengerId']=pd.to_numeric(gender_submission4['PassengerId'])
gender_submission4['Survived']=pd.to_numeric(gender_submission4['Survived'])

In [ ]:
gender_submission4.dtypes

In [ ]:
gender_submission4.shape